# Gerando regras de associação com o A-Priori

Bibliotecas necessárias:

In [1]:
import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
df = pd.read_csv(r'C:/Users/JdMacGyver/Downloads/RR.csv')

In [3]:
df.head(5)

,NT_GER,QE_I01,QE_I02,QE_I03,QE_I04,QE_I05,QE_I06,QE_I07,QE_I08,QE_I09,...,QE_I17,QE_I18,QE_I19,QE_I20,QE_I21,QE_I22,QE_I23,QE_I24,QE_I25,QE_I26
0,25_50,Q01A,Q02A,Q03A,Q04D,Q05B,Q06B,Q07E,Q08B,Q09A,...,Q17A,Q18A,Q19A,Q20H,Q21A,Q22A,Q23E,Q24C,Q25A,Q26A
1,0_25,Q01A,Q02D,Q03A,Q04A,Q05C,Q06C,Q07D,Q08A,Q09F,...,Q17A,Q18B,Q19B,Q20K,Q21A,Q22B,Q23B,Q24D,Q25H,Q26A
2,25_50,Q01A,Q02D,Q03A,Q04B,Q05B,Q06B,Q07C,Q08A,Q09B,...,Q17A,Q18A,Q19B,Q20C,Q21B,Q22E,Q23E,Q24A,Q25H,Q26F
3,25_50,Q01E,Q02A,Q03A,Q04D,Q05D,Q06C,Q07G,Q08C,Q09B,...,Q17A,Q18A,Q19G,Q20K,Q21A,Q22D,Q23E,Q24C,Q25A,Q26A
4,25_50,Q01B,Q02A,Q03A,Q04D,Q05F,Q06C,Q07B,Q08B,Q09F,...,Q17B,Q18A,Q19B,Q20H,Q21A,Q22E,Q23B,Q24A,Q25B,Q26F


#### Passo preliminar: transformar os dados em itemsets (formato de entrada do algoritmo).

In [4]:
te = TransactionEncoder()
te_ary = te.fit(df.values).transform(df.values)
df_ = pd.DataFrame(te_ary, columns=te.columns_)

In [5]:
df_.head(5)

,0_25,25_50,50_75,Q01A,Q01B,Q01C,Q01D,Q01E,Q02A,Q02B,...,Q25G,Q25H,Q26A,Q26B,Q26C,Q26E,Q26F,Q26G,Q26H,Q26I
0,False,True,False,True,False,False,False,False,True,False,...,False,False,True,False,False,False,False,False,False,False
1,True,False,False,True,False,False,False,False,False,False,...,False,True,True,False,False,False,False,False,False,False
2,False,True,False,True,False,False,False,False,False,False,...,False,True,False,False,False,False,True,False,False,False
3,False,True,False,False,False,False,False,True,True,False,...,False,False,True,False,False,False,False,False,False,False
4,False,True,False,False,True,False,False,False,True,False,...,False,False,False,False,False,False,True,False,False,False


# O procedimento de geração de regras de associação é basicamente realizado em dois passos.

#### 1° passo: descobrir os itemsets frequentes.

In [6]:
frequent_itemsets = apriori(df_, min_support=0.4, use_colnames=True)

#### 2° passo: descobrir as regras de associação

In [7]:
rules = association_rules(frequent_itemsets, metric='lift')

In [8]:
rules.head(6)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(25_50),(Q02D),0.638344,0.655773,0.450980,0.706485,1.077330,0.032371,1.172772
1,(Q02D),(25_50),0.655773,0.638344,0.450980,0.687708,1.077330,0.032371,1.158068
2,(25_50),(Q03A),0.638344,0.971678,0.625272,0.979522,1.008073,0.005008,1.383079
3,(Q03A),(25_50),0.971678,0.638344,0.625272,0.643498,1.008073,0.005008,1.014456
4,(25_50),(Q11A),0.638344,0.947712,0.607843,0.952218,1.004755,0.002876,1.094304
5,(Q11A),(25_50),0.947712,0.638344,0.607843,0.641379,1.004755,0.002876,1.008463


In [9]:
asc_rules = rules[['antecedents', 'consequents', 'confidence', 'lift']]

In [10]:
asc_rules.tail(5)

,antecedents,consequents,confidence,lift
18735,(Q03A),"(Q16_14, Q18A, Q15A, Q17A, Q11A, Q14A)",0.419283,0.997154
18736,(Q15A),"(Q16_14, Q18A, Q03A, Q17A, Q11A, Q14A)",0.516575,0.983850
18737,(Q17A),"(Q16_14, Q18A, Q03A, Q15A, Q11A, Q14A)",0.459459,0.967394
18738,(Q11A),"(Q16_14, Q18A, Q03A, Q15A, Q17A, Q14A)",0.429885,1.011883
18739,(Q14A),"(Q16_14, Q18A, Q03A, Q15A, Q17A, Q11A)",0.413717,0.999453


#### Filtro para pegar as regras que envolvem rendimento

In [11]:
notas = set(np.unique(df['NT_GER']))

In [12]:
df_new = pd.DataFrame()

for i in range(len(asc_rules)):
    if notas.intersection(asc_rules['consequents'][i]):
        df_new[i] = asc_rules.loc[i]

df_new = df_new.transpose()

#### Regras Geradas

In [13]:
df_new

,antecedents,consequents,confidence,lift
1,(Q02D),(25_50),0.687708,1.07733
3,(Q03A),(25_50),0.643498,1.00807
5,(Q11A),(25_50),0.641379,1.00475
7,(Q12A),(25_50),0.636872,0.997693
8,(Q14A),(25_50),0.643805,1.00856
11,(Q15A),(25_50),0.629834,0.986669
13,(Q16_14),(25_50),0.657005,1.02923
15,(Q17A),(25_50),0.658477,1.03154
17,(Q18A),(25_50),0.637771,0.999102
18,(Q21A),(25_50),0.62701,0.982244
